In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Data exploration

In [2]:
train_df = pd.read_json("../data/train.json")
valid_df = pd.read_json("../data/valid.json")

In [3]:
len(train_df)

117

In [4]:
#train_df = train_df[:50]

In [5]:
nb_para = 0
for i in range(len(train_df)):
    nb_para += len(train_df.iloc[i]['data']['paragraphs'])
    
nb_para

4921

In [6]:
train_df.head()

,data,version
0,"{'paragraphs': [{'qas': [{'question': ""Quel as...",1
1,{'paragraphs': [{'qas': [{'question': 'America...,1
2,"{'paragraphs': [{'qas': [{'question': ""A quell...",1
3,"{'paragraphs': [{'qas': [{'question': ""Pourquo...",1
4,"{'paragraphs': [{'qas': [{'question': ""Quels s...",1


In [7]:
train_df.iloc[0].keys()

Index(['data', 'version'], dtype='object')

In [8]:
train_df.iloc[0]['data'].keys()

dict_keys(['paragraphs', 'title'])

In [9]:
train_df.iloc[0]['data']

{'paragraphs': [{'qas': [{'question': "Quel astronome a émit l'idée en premier d'une planète entre les orbites de Mars et Jupiter ?",
     'id': '9f38825f-1bd3-4171-9d3b-b0c2c71e7de2',
     'answers': [{'text': 'Johann Elert Bode', 'answer_start': 136}]},
    {'question': 'Quel astronome découvrit Uranus ?',
     'id': 'c2415641-9a62-4052-b57b-9a239da7599c',
     'answers': [{'text': 'William Herschel', 'answer_start': 404}]},
    {'question': 'Quelles furent les découvertes finales des vingt-quatre astronomes ?',
     'id': '5c59e19a-066c-4dc0-aa16-2871dcb12d39',
     'answers': [{'text': 'plusieurs autres astéroïdes',
       'answer_start': 733}]},
    {'question': 'Quelles furent les découvertes finales des vingt-quatre astronomes ?',
     'id': '5c59e19a-066c-4dc0-aa16-2871dcb12d39',
     'answers': [{'text': 'plusieurs autres astéroïdes',
       'answer_start': 733}]}],
   'context': "L'idée selon laquelle une planète inconnue pourrait exister entre les orbites de Mars et Jupiter 

In [10]:
train_df.iloc[0]['data']['paragraphs']

[{'qas': [{'question': "Quel astronome a émit l'idée en premier d'une planète entre les orbites de Mars et Jupiter ?",
    'id': '9f38825f-1bd3-4171-9d3b-b0c2c71e7de2',
    'answers': [{'text': 'Johann Elert Bode', 'answer_start': 136}]},
   {'question': 'Quel astronome découvrit Uranus ?',
    'id': 'c2415641-9a62-4052-b57b-9a239da7599c',
    'answers': [{'text': 'William Herschel', 'answer_start': 404}]},
   {'question': 'Quelles furent les découvertes finales des vingt-quatre astronomes ?',
    'id': '5c59e19a-066c-4dc0-aa16-2871dcb12d39',
    'answers': [{'text': 'plusieurs autres astéroïdes', 'answer_start': 733}]},
   {'question': 'Quelles furent les découvertes finales des vingt-quatre astronomes ?',
    'id': '5c59e19a-066c-4dc0-aa16-2871dcb12d39',
    'answers': [{'text': 'plusieurs autres astéroïdes',
      'answer_start': 733}]}],
  'context': "L'idée selon laquelle une planète inconnue pourrait exister entre les orbites de Mars et Jupiter fut proposée pour la première fois 

In [11]:
print(type(train_df.iloc[0]['data']['paragraphs']))
len(train_df.iloc[0]['data']['paragraphs'])

<class 'list'>


19

In [12]:
train_df.iloc[0]['data']['paragraphs'][0].keys()

dict_keys(['qas', 'context'])

In [13]:
train_df.iloc[0]['data']['paragraphs'][0]

{'qas': [{'question': "Quel astronome a émit l'idée en premier d'une planète entre les orbites de Mars et Jupiter ?",
   'id': '9f38825f-1bd3-4171-9d3b-b0c2c71e7de2',
   'answers': [{'text': 'Johann Elert Bode', 'answer_start': 136}]},
  {'question': 'Quel astronome découvrit Uranus ?',
   'id': 'c2415641-9a62-4052-b57b-9a239da7599c',
   'answers': [{'text': 'William Herschel', 'answer_start': 404}]},
  {'question': 'Quelles furent les découvertes finales des vingt-quatre astronomes ?',
   'id': '5c59e19a-066c-4dc0-aa16-2871dcb12d39',
   'answers': [{'text': 'plusieurs autres astéroïdes', 'answer_start': 733}]},
  {'question': 'Quelles furent les découvertes finales des vingt-quatre astronomes ?',
   'id': '5c59e19a-066c-4dc0-aa16-2871dcb12d39',
   'answers': [{'text': 'plusieurs autres astéroïdes', 'answer_start': 733}]}],
 'context': "L'idée selon laquelle une planète inconnue pourrait exister entre les orbites de Mars et Jupiter fut proposée pour la première fois par Johann Elert Bo

In [14]:
train_df.iloc[0]['data']['paragraphs'][0]['context']

"L'idée selon laquelle une planète inconnue pourrait exister entre les orbites de Mars et Jupiter fut proposée pour la première fois par Johann Elert Bode en 1768. Ses suggestions étaient basées sur la loi de Titius-Bode, une théorie désormais obsolète proposée par Johann Daniel Titius en 1766,. Selon cette loi, le demi-grand axe de cette planète aurait été d'environ 2,8 ua. La découverte d'Uranus par William Herschel en 1781 accrut la confiance dans la loi de Titius-Bode et, en 1800, vingt-quatre astronomes expérimentés combinèrent leurs efforts et entreprirent une recherche méthodique de la planète proposée,. Le groupe était dirigé par Franz Xaver von Zach. Bien qu'ils n'aient pas découvert Cérès, ils trouvèrent néanmoins plusieurs autres astéroïdes."

Organization of the data :

Every dataframe element contains several paragraphs of a same text.
Every paragraph contains a list of questions / answers and the complete text of the paragraph.

## OKAPI BM25

### Process the paragraphs

In [15]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('french'))

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nassy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def remove_stopwords(text):
    stop = stopwords.words('french')
    ans = []
    
    for word in text:
        if not word in stop:
            for char in [',', '.', ':', ';']:
                word = word.replace(char, '')
            ans.append(word)
            
    return ans

In [17]:
# Create a list of documents
contexts = []

for i in range(len(train_df)):
    for j in range(len(train_df.iloc[i]['data']['paragraphs'])):
        text = remove_stopwords(train_df.iloc[i]['data']['paragraphs'][j]['context'].split())
        contexts.append(text)

In [18]:
contexts[0]

["L'idée",
 'selon',
 'laquelle',
 'planète',
 'inconnue',
 'pourrait',
 'exister',
 'entre',
 'orbites',
 'Mars',
 'Jupiter',
 'proposée',
 'première',
 'fois',
 'Johann',
 'Elert',
 'Bode',
 '1768',
 'Ses',
 'suggestions',
 'basées',
 'loi',
 'Titius-Bode',
 'théorie',
 'désormais',
 'obsolète',
 'proposée',
 'Johann',
 'Daniel',
 'Titius',
 '1766',
 'Selon',
 'cette',
 'loi',
 'demi-grand',
 'axe',
 'cette',
 'planète',
 "d'environ",
 '28',
 'ua',
 'La',
 'découverte',
 "d'Uranus",
 'William',
 'Herschel',
 '1781',
 'accrut',
 'confiance',
 'loi',
 'Titius-Bode',
 'et',
 '1800',
 'vingt-quatre',
 'astronomes',
 'expérimentés',
 'combinèrent',
 'leurs',
 'efforts',
 'entreprirent',
 'recherche',
 'méthodique',
 'planète',
 'proposée',
 'Le',
 'groupe',
 'dirigé',
 'Franz',
 'Xaver',
 'von',
 'Zach',
 'Bien',
 "qu'ils",
 "n'aient",
 'découvert',
 'Cérès',
 'trouvèrent',
 'néanmoins',
 'plusieurs',
 'autres',
 'astéroïdes']

In [19]:
# Stemming the text
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("french")

contexts = [[stemmer.stem(word) for word in context] for context in contexts]

In [20]:
contexts[0]

["l'id",
 'selon',
 'laquel',
 'planet',
 'inconnu',
 'pourr',
 'exist',
 'entre',
 'orbit',
 'mar',
 'jupit',
 'propos',
 'premi',
 'fois',
 'johann',
 'elert',
 'bod',
 '1768',
 'se',
 'suggest',
 'bas',
 'loi',
 'titius-bod',
 'théor',
 'désorm',
 'obsolet',
 'propos',
 'johann',
 'daniel',
 'titius',
 '1766',
 'selon',
 'cet',
 'loi',
 'demi-grand',
 'axe',
 'cet',
 'planet',
 "d'environ",
 '28',
 'ua',
 'la',
 'découvert',
 "d'uranus",
 'william',
 'herschel',
 '1781',
 'accrut',
 'confianc',
 'loi',
 'titius-bod',
 'et',
 '1800',
 'vingt-quatr',
 'astronom',
 'expériment',
 'combin',
 'leur',
 'effort',
 'entrepr',
 'recherch',
 'méthod',
 'planet',
 'propos',
 'le',
 'group',
 'dirig',
 'franz',
 'xav',
 'von',
 'zach',
 'bien',
 "qu'il",
 "n'aient",
 'découvert',
 'céres',
 'trouv',
 'néanmoin',
 'plusieur',
 'autr',
 'astéroïd']

In [21]:
# Change contexts into a bags of words

bows = []

for text in contexts:
    bows.append({})
    for word in text:
         bows[-1][word] = bows[-1].get(word, 0) + 1

In [22]:
# Show an example of a bag of words
bows[0]

{"l'id": 1,
 'selon': 2,
 'laquel': 1,
 'planet': 3,
 'inconnu': 1,
 'pourr': 1,
 'exist': 1,
 'entre': 1,
 'orbit': 1,
 'mar': 1,
 'jupit': 1,
 'propos': 3,
 'premi': 1,
 'fois': 1,
 'johann': 2,
 'elert': 1,
 'bod': 1,
 '1768': 1,
 'se': 1,
 'suggest': 1,
 'bas': 1,
 'loi': 3,
 'titius-bod': 2,
 'théor': 1,
 'désorm': 1,
 'obsolet': 1,
 'daniel': 1,
 'titius': 1,
 '1766': 1,
 'cet': 2,
 'demi-grand': 1,
 'axe': 1,
 "d'environ": 1,
 '28': 1,
 'ua': 1,
 'la': 1,
 'découvert': 2,
 "d'uranus": 1,
 'william': 1,
 'herschel': 1,
 '1781': 1,
 'accrut': 1,
 'confianc': 1,
 'et': 1,
 '1800': 1,
 'vingt-quatr': 1,
 'astronom': 1,
 'expériment': 1,
 'combin': 1,
 'leur': 1,
 'effort': 1,
 'entrepr': 1,
 'recherch': 1,
 'méthod': 1,
 'le': 1,
 'group': 1,
 'dirig': 1,
 'franz': 1,
 'xav': 1,
 'von': 1,
 'zach': 1,
 'bien': 1,
 "qu'il": 1,
 "n'aient": 1,
 'céres': 1,
 'trouv': 1,
 'néanmoin': 1,
 'plusieur': 1,
 'autr': 1,
 'astéroïd': 1}

### Process the questions and answers

In [23]:
questions = []
answers = []
id_to_ans = {}
for i in range(len(train_df)):
    for j in range(len(train_df.iloc[i]['data']['paragraphs'])):
        for k in range(len(train_df.iloc[i]['data']['paragraphs'][j]['qas'])):
            questions.append(train_df.iloc[i]['data']['paragraphs'][j]['qas'][k]['question'])
            answers.append([i,j])
        id_to_ans[(i,j)] = len(id_to_ans)
            

questions = [[stemmer.stem(word) for word in remove_stopwords(question.split())] for question in questions]

In [24]:
import random

random.Random(42).shuffle(questions)
random.Random(42).shuffle(answers)

**We will test the shuffling.**

In [25]:
print(questions[:5], '\n', answers[:5])

[['qui', 'a', 'accept', 'product', 'dumbo', 'comm', 'long', 'métrag', '?'], ['quel', 'joueur', 'vient', 'complet', "l'attaqu", 'maroon', 'plac', 'siebert', '?'], ['qui', 'gagn', 'championnat', 'mond', 'goodwood', '?'], ['quel', 'deux', 'surnom', "l'antarct", '?'], ['quel', 'plus', 'ancien', 'couleur', 'rob', 'persan', '?']] 
 [[30, 9], [68, 5], [10, 17], [4, 0], [89, 8]]


In [26]:
train_df.iloc[8]['data']['paragraphs'][23]

{'qas': [{'question': "Comment s'appelle le bâtiment fortifié près du port ? ",
   'id': 'b315f275-ecc2-4069-a4b5-0f4a6037a9cc',
   'answers': [{'text': "l'abbaye de Saint-Victor", 'answer_start': 62}]},
  {'question': "Comment s'appelle le duc de Savoie en 1591 ? ",
   'id': '4eda5e4b-f6ab-493c-9fc8-fa7cd86f3b8c',
   'answers': [{'text': 'Charles Emmanuel', 'answer_start': 9}]},
  {'question': 'Quel poste occupe Pierre Bon à Notre-Dame de la Garde ? ',
   'id': 'e37fb819-f53c-448c-a92f-818c9cc0af6d',
   'answers': [{'text': 'gouverneur', 'answer_start': 160}]},
  {'question': "A quelle date le sieur Méolhon parvient à occuper l'abbaye ? ",
   'id': '69cf69bd-260c-40bc-8510-58ac7a00ba7f',
   'answers': [{'text': 'la nuit du 16 novembre 1591', 'answer_start': 229}]},
  {'question': "Qui parvient à reprendre l'abbaye au sieur Méolhon ? ",
   'id': '7e979854-2122-4833-a446-bf2a57fedeb8',
   'answers': [{'text': 'les partisans de Charles de Casaulx',
     'answer_start': 327}]}],
 'context

**The answers and questions are shuffled the same way.**

### Preparing the OKAPI BM25 formula

In [27]:
import math

NB_DOC = 0

for i in range(len(train_df)):
    for _ in range(len(train_df.iloc[i]['data']['paragraphs'])):
        NB_DOC += 1

k1 = 1.6
b = 0.75

def bow_len(bow):
    return sum([bow[key] for key in bow])

def average(lst): 
    return sum(lst) / len(lst) 

avg_dl = average([bow_len(bow) for bow in bows])

def freq_in_doc(word, doc_bow):
    """
    Returns the number of iterations of a given word in a bag of words.
    """
    return doc_bow.get(word, 0)

def nb_of_docs(word):
    """
    Returns the number of documents containing a given word.
    """
    ans = 0
    
    for bow in bows:
        if word in bow:
            ans += 1
    
    return ans

def idf(word):
    """
    Computes the idf of a word in the entire corpus.
    """
    docs_nb = nb_of_docs(word)
    return math.log((NB_DOC - docs_nb + 0.5) / (docs_nb + 0.5) + 1)

def single_okapi(word, bow):
    """
    Computes the OKAPI BM25 score relative to a single word.
    """
    freq = freq_in_doc(word, bow)
    nb_of_words = bow_len(bow)
    
    ans = idf(word) * (k1 + 1) * freq / (freq + k1 * (1 - b + b * nb_of_words / avg_dl))
    
    return ans

def okapi_score(question, bow):
    """
    Computes the final OKAPI BM25 score relative to an entire question.
    """
    return sum([single_okapi(word, bow) for word in question])

In [28]:
question = questions[0]
print(question)
print(okapi_score(question, bows[id_to_ans[(18,14)]]))

['qui', 'a', 'accept', 'product', 'dumbo', 'comm', 'long', 'métrag', '?']
0.0


In [29]:
answers[0]

[30, 9]

In [30]:
def get_best_fit(question):
    best_score = 0
    best_bow = 0
    for i in range(len(bows)):
        score = okapi_score(question, bows[i])
        
        if score > best_score:
            best_bow = i
            best_score = score
            
    return best_bow

def get_best_fits(question, propositions, text = False):
    scores = []
    for i in range(len(bows)):
        scores.append([i, okapi_score(question, bows[i])])

    scores.sort(key = (lambda x: -x[1]))
    
    if text:
        return [(scores[i][1], ' '.join(bows[scores[i][0]].keys())) for i in range(propositions)]

    return scores[:propositions]

### Testing the method

In [31]:
get_best_fits(question, 5)

[[1109, 26.496052514308325],
 [1102, 24.855077045111678],
 [412, 18.512200357543527],
 [2568, 17.360184845503714],
 [428, 17.07298847281217]]

In [32]:
get_best_fits(question, 5, text = True)

[(26.496052514308325,
  "apres avoir accept product dumbo comm long métrag walt a dû fair fac effet ordre chronolog grev studios disney (à part fin mai 1941) voyag amer latin mi-août l'entr guerr états-un on peut auss parl impliqu bamb (1942) dont « croi davantag succes » réalis plus compliqu raison difficult anim façon réalist animal"),
 (24.855077045111678,
  "pierr lambert écrit que malgr l'id premi walt disney réalis long métrag d'anim budget réduit peu intervenu personnel product dumbo d'autr raison budgétair toutefois richard shickel film inaugur princip repr plus tard alic pay merveil (1951) pet pan (1953) celui d'un comm court"),
 (18.512200357543527,
  "autr élément li nécess trouv fond walt disney abandon droit publiqu musiqu blanche-neig l'éditeur bourn music company détient encor jour ains musical pinocchio dumbo apres mort saul 1957 jimmy johnson têt buen vist record a essai deux fois rachet product anné 1930 1940 premi long métrag silly symphony san succes la veuv refus d

In [33]:
id_to_ans[(18,14)]

688

In [34]:
correct = 0
pool_size = 5

iterator = tqdm(range(len(questions)))

for i in iterator:
    question = questions[i]
    #good_ids = [get_best_fits(question, pool_size)[i][0] for i in range(pool_size)]
    
    #if id_to_ans[tuple(answers[i])] in good_ids:
        #correct += 1
        
    good_id = get_best_fits(question, 1)[0][0]
        
    if good_id == id_to_ans[tuple(answers[i])]:
        correct += 1
        
    iterator.set_description(str(correct/(i+1)))
        
correct / len(questions)

  0%|          | 0/20731 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
correct = 0
pool_size = 10

iterator = tqdm(range(len(questions)))

for i in iterator:
    question = questions[i]
    good_ids = [get_best_fits(question, pool_size)[i][0] for i in range(pool_size)]
    
    if id_to_ans[tuple(answers[i])] in good_ids:
        correct += 1
        
    #good_id = get_best_fits(question, 1)[0][0]
        
    #if good_id == id_to_ans[tuple(answers[i])]:
        #correct += 1
        
    iterator.set_description(str(correct/(i+1)))
        
correct / len(questions)

### Reducing the complexity

In [35]:
idf_store = {}

for question in questions:
    for word in question:
        if word in idf_store:
            pass
        else:
            idf_store[word] = idf_store.get(word, idf(word))

In [36]:
def single_okapi_opti(word, bow):
    """
    Computes the OKAPI BM25 score relative to a single word.
    """
    freq = freq_in_doc(word, bow)
    nb_of_words = bow_len(bow)
    
    ans = idf_store[word] * (k1 + 1) * freq / (freq + k1 * (1 - b + b * nb_of_words / avg_dl))
    
    return ans

def okapi_score_opti(question, bow):
    """
    Computes the final OKAPI BM25 score relative to an entire question.
    """
    return sum([single_okapi_opti(word, bow) for word in question])

def get_best_fits_opti(question, propositions, text = False):
    scores = []
    for i in range(len(bows)):
        scores.append([i, okapi_score_opti(question, bows[i])])

    scores.sort(key = (lambda x: -x[1]))
    
    if text:
        return [(scores[i][1], ' '.join(bows[scores[i][0]].keys())) for i in range(propositions)]

    return scores[:propositions]

In [37]:
correct = 0
pool_size = 5

iterator = tqdm(range(len(questions)))

for i in iterator:
    question = questions[i]
    #good_ids = [get_best_fits(question, pool_size)[i][0] for i in range(pool_size)]
    
    #if id_to_ans[tuple(answers[i])] in good_ids:
        #correct += 1
        
    good_id = get_best_fits_opti(question, 1)[0][0]
        
    if good_id == id_to_ans[tuple(answers[i])]:
        correct += 1
        
    iterator.set_description(str(correct/(i+1)))
        
correct / len(questions)

0.4830447156432396: 100%|██████████| 20731/20731 [1:04:50<00:00,  5.33it/s] 


0.4830447156432396

In [ ]:
correct = 0
pool_size = 10

iterator = tqdm(range(len(questions)))

for i in iterator:
    question = questions[i]
    good_ids = [get_best_fits_opti(question, pool_size)[i][0] for i in range(pool_size)]
    
    if id_to_ans[tuple(answers[i])] in good_ids:
        correct += 1
        
    #good_id = get_best_fits(question, 1)[0][0]
        
    #if good_id == id_to_ans[tuple(answers[i])]:
        #correct += 1
        
    iterator.set_description(str(correct/(i+1)))
        
correct / len(questions)

This method is now 20 to 30 times faster.

## Using CamemBERT

In [38]:
from transformers import CamembertForQuestionAnswering, CamembertTokenizer

C:\Users\nassy\Anaconda3\envs\PyTorch\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nassy\Anaconda3\envs\PyTorch\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nassy\Anaconda3\envs\PyTorch\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nassy\Anaconda3\envs\PyTorch\lib\site

In [39]:
model = CamembertForQuestionAnswering.from_pretrained("illuin/camembert-base-fquad")
tokenizer = CamembertTokenizer.from_pretrained("illuin/camembert-base-fquad")

In [40]:
# Create a list of documents
contexts = []

for i in range(len(train_df)):
    for j in range(len(train_df.iloc[i]['data']['paragraphs'])):
        text = train_df.iloc[i]['data']['paragraphs'][j]['context']
        contexts.append(text)

In [41]:
encoded_context = [tokenizer.encode(contexts[i]) for i in range(len(contexts))]

In [42]:
questions = []
answers = []
id_to_ans = {}
for i in range(len(train_df)):
    for j in range(len(train_df.iloc[i]['data']['paragraphs'])):
        for k in range(len(train_df.iloc[i]['data']['paragraphs'][j]['qas'])):
            questions.append(tokenizer.encode(train_df.iloc[i]['data']['paragraphs'][j]['qas'][k]['question']))
            answers.append([i,j])
        id_to_ans[(i,j)] = len(id_to_ans)